In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification


from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler



> Raw Data

In [2]:
csv_file = r"D:\ineuron\Accenture Task\scrapper\gas_data_dict.csv"

df = pd.read_csv(csv_file)
df.head()

,Feature class,Feature class element,Geometry type
0,PipelineAssembly,Compressor Station,polygon
1,PipelineAssembly,Fitting Assembly,polygon
2,PipelineAssembly,Meter Setting,polygon
3,PipelineAssembly,Pigging Structure,polygon
4,PipelineAssembly,Pump Station,polygon


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Feature class          152 non-null    object
 1   Feature class element  152 non-null    object
 2   Geometry type          152 non-null    object
dtypes: object(3)
memory usage: 3.7+ KB


In [4]:
df.describe()

,Feature class,Feature class element,Geometry type
count,152,152,152
unique,8,146,4
top,Attribute Domains,Unknown,attribute
freq,74,7,74


> precess data

In [5]:
# Depenedent and independet variable

# Split data into features and target variable
X = df[['Feature class element', 'Geometry type']]
y = df[['Feature class']]


X.shape , y.shape

((152, 2), (152, 1))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_test.to_csv('X_test.csv', index=False)
y_test.to_csv('y_test.csv',  index=False)

> Encoding

In [7]:
# One-hot encode categorical variables
cat_x = ['Feature class element', 'Geometry type']
target_column = ['Feature class']

# data_encoded = pd.get_dummies(df, columns=cat_x)


cat_pipeline=Pipeline(
    
    steps=[
        
        ("imputer",SimpleImputer(strategy="most_frequent")),
        ("encoder",OneHotEncoder())
    ]


)

preprocessor=ColumnTransformer([
    ('cat_x',cat_pipeline,cat_x)
])


In [8]:
x_train = preprocessor.fit_transform(X)

In [9]:
y_train = y

# Random Forest

In [10]:
model = RandomForestClassifier(oob_score=True)

In [11]:
model.fit(x_train , y_train)

C:\Users\aaron\AppData\Local\Temp\ipykernel_2140\1964293123.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train , y_train)


RandomForestClassifier(oob_score=True)

In [12]:
# loading x_test and y_test data
x_for_test = pd.read_csv(r"D:\ineuron\Accenture Task\model\X_test.csv")
y_test = pd.read_csv(r"D:\ineuron\Accenture Task\model\y_test.csv")



> prediction

In [13]:
X_test=preprocessor.transform(x_for_test)
y_pred = model.predict(X_test)

y_pred

array(['StructureJunction', 'Attribute Domains', 'Attribute Domains',
       'Attribute Domains', 'Attribute Domains', 'Attribute Domains',
       'PipelineDevice', 'PipelineDevice', 'PipelineDevice',
       'PipelineDevice', 'Attribute Domains', 'Attribute Domains',
       'Attribute Domains', 'PipelineDevice', 'StructureJunction',
       'PipelineAssembly', 'StructureBoundary', 'PipelineJunction',
       'Attribute Domains', 'Attribute Domains', 'Attribute Domains',
       'PipelineJunction', 'PipelineLine', 'PipelineAssembly',
       'PipelineLine', 'Attribute Domains', 'PipelineJunction',
       'Attribute Domains', 'Attribute Domains', 'PipelineJunction',
       'Attribute Domains'], dtype=object)

In [14]:
ypred = pd.DataFrame(y_pred , columns=['Feature class predicted'])

ypred.head()

,Feature class predicted
0,StructureJunction
1,Attribute Domains
2,Attribute Domains
3,Attribute Domains
4,Attribute Domains


In [15]:
df_for_export = pd.concat([x_for_test ,y_test, ypred], axis=1, ignore_index=False)


In [17]:
df_for_export.head()

,Feature class element,Geometry type,Feature class,Feature class predicted
0,Pipeline Anchor,point,StructureJunction,StructureJunction
1,Pipeline Valve Type,attribute,Attribute Domains,Attribute Domains
2,Pipeline ASTM Material,attribute,Attribute Domains,Attribute Domains
3,CP ASTM Anode Manufacture Component,attribute,Attribute Domains,Attribute Domains
4,Pipeline Pipe Wall Thickness - Transmission,attribute,Attribute Domains,Attribute Domains


In [16]:
df_for_export.to_csv("pred_output.csv" , index=False)

In [ ]:

y_test.columns

Index(['Feature class'], dtype='object')

In [ ]:
y_test.head()

,Feature class
0,StructureJunction
1,Attribute Domains
2,Attribute Domains
3,Attribute Domains
4,Attribute Domains


In [ ]:
from sklearn.metrics import accuracy_score
accuracy=round(accuracy_score(y_test,y_pred)*100,2)
accuracy

96.77